In [1]:
# Package installs
%pip install pandas
%pip install matplotlib
%pip install numpy
%pip install seaborn
%pip install scikit-learn
%pip install cartopy
%pip install keras
%pip install keras-tuner
# you may have to install in console using pip install tensorflow --user or pip install tensorflow --upgrade
%pip install tensorflow
%pip install basketball-reference-scraper

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

In [2]:
%pip install nba_api

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import datetime
import time
import basketball_reference_scraper
# Modeling tools
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
import keras_tuner as kt
from basketball_reference_web_scraper import client
from basketball_reference_web_scraper.data import OutputType
from datetime import date
from nba_api.stats.endpoints import leaguegamelog

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.metrics import MeanAbsoluteError
import keras_tuner as kt
import joblib

In [3]:
player_stats = client.players_season_totals(season_end_year=2025)

playoff_games = leaguegamelog.LeagueGameLog(season='2023-24', season_type_all_star='Playoffs')
df_playoffs = playoff_games.get_data_frames()[0]
print(player_stats)  # List of all players' season stats

[{'slug': 'gilgesh01', 'name': 'Shai Gilgeous-Alexander', 'positions': [<Position.POINT_GUARD: 'POINT GUARD'>], 'age': 26, 'team': <Team.OKLAHOMA_CITY_THUNDER: 'OKLAHOMA CITY THUNDER'>, 'games_played': 76, 'games_started': 76, 'minutes_played': 2598, 'made_field_goals': 860, 'attempted_field_goals': 1656, 'made_three_point_field_goals': 163, 'attempted_three_point_field_goals': 435, 'made_free_throws': 601, 'attempted_free_throws': 669, 'offensive_rebounds': 67, 'defensive_rebounds': 312, 'assists': 486, 'steals': 131, 'blocks': 77, 'turnovers': 183, 'personal_fouls': 164, 'points': 2484}, {'slug': 'edwaran01', 'name': 'Anthony Edwards', 'positions': [<Position.SHOOTING_GUARD: 'SHOOTING GUARD'>], 'age': 23, 'team': <Team.MINNESOTA_TIMBERWOLVES: 'MINNESOTA TIMBERWOLVES'>, 'games_played': 79, 'games_started': 79, 'minutes_played': 2871, 'made_field_goals': 721, 'attempted_field_goals': 1612, 'made_three_point_field_goals': 320, 'attempted_three_point_field_goals': 811, 'made_free_throws'

In [4]:
df1 = pd.DataFrame(player_stats)

In [5]:
df2 = pd.DataFrame(player_stats)

# Calculate per-game averages
df2["points_per_game"] = df2["points"] / df2["games_played"]
df2["assists_per_game"] = df2["assists"] / df2["games_played"]
df2["total_rebounds"] = df2["offensive_rebounds"] + df2["defensive_rebounds"]
df2["rebounds_per_game"] = df2["total_rebounds"] / df2["games_played"]
df2["steals_per_game"] = df2["steals"] / df2["games_played"]
df2["blocks_per_game"] = df2["blocks"] / df2["games_played"]
df2["turnovers_per_game"] = df2["turnovers"] / df2["games_played"]

# Display only relevant columns
df2 = df2[["name", "points_per_game", "assists_per_game", "rebounds_per_game","blocks_per_game" , "turnovers_per_game"]]

In [6]:
df2.to_csv("average_stats.csv", index=False)

In [7]:
all_games_2025 = client.season_schedule(season_end_year=2025)
df_2025 = pd.DataFrame(all_games_2025)

csv_filename = "NBA_2025_schedule.csv"
teamlist = []
for team in df_2025['away_team']:
    add = True
    for i in teamlist:
        if team == i:
            add = False
    if(add):        
        teamlist.append(team)

df_tlist =pd.DataFrame(teamlist)
df_tlist.to_csv("ListOfTeams.csv",index = False)
df_2025.to_csv( "NBA_2025_schedule.csv", index = False)      

In [8]:
df = pd.read_csv("NBA_2025_schedule.csv")


df = df.dropna(subset=['away_team_score'])

# Save the cleaned DataFrame
df.to_csv("NBA_2025_cleaned_schedule.csv", index=False)
        
        


'''df['away_team'] = df['away_team'].str.replace("TEAM.", '', regex=True)
df['home_team'] = df['home_team'].str.replace("TEAM.", "Bruh", regex=True)
df.to_csv( "NBA_20256_schedule.csv", index = False)'''      

'df[\'away_team\'] = df[\'away_team\'].str.replace("TEAM.", \'\', regex=True)\ndf[\'home_team\'] = df[\'home_team\'].str.replace("TEAM.", "Bruh", regex=True)\ndf.to_csv( "NBA_20256_schedule.csv", index = False)'

In [9]:
df = pd.read_csv("NBA_2025_cleaned_schedule.csv")

df['away_team'] = df['away_team'].str.replace('Team.','',)
df['home_team'] = df['home_team'].str.replace('Team.','',)

df.to_csv("NBA_2025_OFFICIAL_TEAM_SCHEDULE.csv", index=False)

In [10]:
df = pd.read_csv("NBA_2025_OFFICIAL_TEAM_SCHEDULE.csv")
df1 = pd.read_csv("ListOfTeams.csv")
df['TEAMS'] = df1

df.to_csv("NBA_2025_OFFICIAL_TEAM_SCHEDULE.csv", index=False)

In [11]:

df = pd.read_csv("NBA_2025_OFFICIAL_TEAM_SCHEDULE.csv")
calcdf = []

for i in range(len(df)):
    home_score = df.at[i, 'home_team_score']
    away_score = df.at[i, 'away_team_score']
    home_team = df.at[i, 'home_team']
    away_team = df.at[i, 'away_team']

    home_won = 1 if home_score > away_score else 0
    away_won = 1 if away_score > home_score else 0

    # Append home team results
    calcdf.append({
        'TEAM': home_team,
        'OPPONENT': away_team,
        'TEAM_HOME': 1,  # 1 = Home Game
        'WIN': home_won,
        'TEAM_SCORE': home_score,
        'OPPONENT_SCORE': away_score
    })

    # Append away team results
    calcdf.append({
        'TEAM': away_team,
        'OPPONENT': home_team,
        'TEAM_HOME': 0,  # 0 = Away Game
        'WIN': away_won,
        'TEAM_SCORE': away_score,
        'OPPONENT_SCORE': home_score
    })

# Convert to DataFrame
dflonger = pd.DataFrame(calcdf)

# Group and summarize matchups properly
dfsummary = dflonger.groupby(['TEAM', 'OPPONENT', 'TEAM_HOME']).agg(
    games_played=('WIN', 'size'),
    wins=('WIN', 'sum'),
    total_points_scored=('TEAM_SCORE', 'sum'),
    total_points_allowed=('OPPONENT_SCORE', 'sum'),
    avg_points_scored=('TEAM_SCORE', 'mean'),
    avg_points_allowed=('OPPONENT_SCORE', 'mean')
).reset_index()

# Save to CSV
dfsummary.to_csv("NBA_Team_Performance.csv", index=False)

# Display summary
print(dfsummary)

                    TEAM           OPPONENT  TEAM_HOME  games_played  wins  \
0          ATLANTA_HAWKS     BOSTON_CELTICS          0             2     2   
1          ATLANTA_HAWKS     BOSTON_CELTICS          1             1     0   
2          ATLANTA_HAWKS      BROOKLYN_NETS          0             2     1   
3          ATLANTA_HAWKS      BROOKLYN_NETS          1             1     1   
4          ATLANTA_HAWKS  CHARLOTTE_HORNETS          0             2     2   
...                  ...                ...        ...           ...   ...   
1735  WASHINGTON_WIZARDS  SAN_ANTONIO_SPURS          1             1     0   
1736  WASHINGTON_WIZARDS    TORONTO_RAPTORS          0             2     1   
1737  WASHINGTON_WIZARDS    TORONTO_RAPTORS          1             2     0   
1738  WASHINGTON_WIZARDS          UTAH_JAZZ          0             1     0   
1739  WASHINGTON_WIZARDS          UTAH_JAZZ          1             1     1   

      total_points_scored  total_points_allowed  avg_points_sco

In [12]:
gamelog = leaguegamelog.LeagueGameLog(season='2024-25', player_or_team_abbreviation='P')
player_logs = gamelog.get_data_frames()[0]
gamelog_playoffs = leaguegamelog.LeagueGameLog(season='2024-25', season_type_all_star='Playoffs', player_or_team_abbreviation='P')
player_logs_playoffs = gamelog_playoffs.get_data_frames()[0]


full_logs = pd.concat([player_logs, player_logs_playoffs])
full_logs.to_csv("NBA_Box_score_24-25_combined.csv", index=False)

print(player_logs[['PLAYER_NAME', 'GAME_DATE', 'TEAM_ABBREVIATION', 'MATCHUP', 'PTS']].head())

player_logs.to_csv("NBA_Box_score_24-25.csv", index=False)

                PLAYER_NAME   GAME_DATE TEAM_ABBREVIATION      MATCHUP  PTS
0             Jalen Brunson  2024-10-22               NYK    NYK @ BOS   22
1             Dalton Knecht  2024-10-22               LAL  LAL vs. MIN    5
2           Jaden McDaniels  2024-10-22               MIN    MIN @ LAL    6
3                  Naz Reid  2024-10-22               MIN    MIN @ LAL   12
4  Nickeil Alexander-Walker  2024-10-22               MIN    MIN @ LAL   14


In [13]:
teamlog = leaguegamelog.LeagueGameLog(season='2023-24', player_or_team_abbreviation='T')  
team_logs = teamlog.get_data_frames()[0]

In [14]:
print(player_logs['GAME_DATE'].max())

2025-04-13


In [15]:
box_scores = pd.read_csv("NBA_Box_Score_24-25_combined.csv")
player_avg = pd.read_csv("average_stats.csv")
team_perf = pd.read_csv("NBA_Team_Performance.csv")

In [16]:
box_scores['HOME'] = box_scores['MATCHUP'].str.contains(r'vs\.').astype(int)

box_scores['OPPONENT'] = box_scores['MATCHUP'].str.extract(r'vs\. (\w+)|@ (\w+)').bfill(axis=1).iloc[:, 0]

box_scores['TEAM'] = box_scores['TEAM_NAME'].str.upper().str.replace(" ", "_")
box_scores['OPPONENT'] = box_scores['OPPONENT'].str.upper()
box_scores['GAME_DATE'] = pd.to_datetime(box_scores['GAME_DATE'])

box_scores = box_scores.sort_values(['PLAYER_NAME', 'GAME_DATE'])

for stat in ['PTS', 'AST', 'REB']:
    box_scores[f'{stat}_ROLL3'] = box_scores.groupby('PLAYER_NAME')[stat].rolling(3, min_periods=1).mean().reset_index(0, drop=True)

In [17]:
player_avg.rename(columns={
    'name': 'PLAYER_NAME',
    'points_per_game': 'PPG',
    'assists_per_game': 'APG',
    'rebounds_per_game': 'RPG'
}, inplace=True)

full_df = box_scores.merge(player_avg, how='left', on='PLAYER_NAME')

In [18]:
team_perf['TEAM'] = team_perf['TEAM'].str.upper().str.replace(" ", "_")
team_perf['OPPONENT'] = team_perf['OPPONENT'].str.upper().str.replace(" ", "_")

full_df = full_df.merge(
    team_perf[['TEAM', 'OPPONENT', 'TEAM_HOME', 'avg_points_scored', 'avg_points_allowed']],
    how='left',
    left_on=['TEAM', 'OPPONENT', 'HOME'],
    right_on=['TEAM', 'OPPONENT', 'TEAM_HOME']
)

In [19]:
full_df.to_csv("structured_betting_dataset.csv", index=False)

In [20]:
#Drop empty values in structured betting data
full_df[['avg_points_scored', 'avg_points_allowed']] = full_df[['avg_points_scored', 'avg_points_allowed']].fillna(-1)

In [21]:
full_df.to_csv("structured_betting_dataset.csv", index=False)

In [22]:
full_df['avg_points_scored'] = full_df['avg_points_scored'].fillna(-1.0)
full_df['avg_points_allowed'] = full_df['avg_points_allowed'].fillna(-1.0)

In [23]:
full_df.to_csv("structured_betting_dataset.csv", index=False)

In [24]:
full_df['MATCHUP_KNOWN'] = (full_df['avg_points_scored'] != -1.0).astype(int)

In [25]:
full_df.to_csv("structured_betting_dataset.csv", index=False)

In [26]:

def r2_metric(y_true, y_pred):
    SS_res = np.sum(np.square(y_true - y_pred))
    SS_tot = np.sum(np.square(y_true - np.mean(y_true)))
    return 1 - SS_res / (SS_tot + 1e-7)

'''def lr_schedule(epoch, lr):
    if epoch <= 10:
        if epoch % 2 == 0 and epoch > 0:
            return lr * 0.9
    else:
        return lr * 0.9
    return lr'''


def build_rnn_sequences(df, player_name, stat, threshold, features, window=5):
    df_player = df[df['PLAYER_NAME'] == player_name].copy().sort_values("GAME_DATE")
    df_player['TARGET'] = (df_player[stat] > threshold).astype(int)

    scaler = StandardScaler()
    df_player[features] = scaler.fit_transform(df_player[features])

    X, y = [], []
    for i in range(len(df_player) - window):
        seq = df_player[features].iloc[i:i+window].values
        label = df_player['TARGET'].iloc[i+window]
        X.append(seq)
        y.append(label)

    return np.array(X), np.array(y), scaler
class NeuralNetwork:
    def __init__(self, X_train, X_test, y_train, y_test):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.model = None

    def build_model(self, input_shape):
        self.model = Sequential()
        self.model.add(LSTM(64, return_sequences=True, input_shape=input_shape))
        self.model.add(LSTM(32, return_sequences=False))
        self.model.add(Dense(25, activation='relu'))
        self.model.add(Dense(1, activation='sigmoid'))

        self.model.compile(optimizer='adam',
                           loss='binary_crossentropy',
                           metrics=['accuracy', MeanAbsoluteError()])

        self.model.summary()

    def train(self, epochs=30, batch_size=32):
        #scheduler = LearningRateScheduler(lr_schedule)
        self.model.fit(self.X_train, self.y_train,
               validation_data=(self.X_test, self.y_test),
               epochs=epochs,
               batch_size=batch_size)
        self.model.save("rnn_player_model.h5")
        print("Model saved as 'rnn_player_model.h5'")

    def evaluate(self):
        results = self.model.evaluate(self.X_test, self.y_test)
        print(f"Loss: {results[0]}, Accuracy: {results[1]}, MAE: {results[2]}")
        
if __name__ == "__main__":

    df = pd.read_csv("structured_betting_dataset.csv")

    player = "Stephen Curry"
    stat = "AST"
    threshold = 4

    features = ['PTS_ROLL3', 'AST_ROLL3', 'REB_ROLL3', 'PPG', 'APG', 'RPG', 'HOME', 'avg_points_allowed']

    X, y, scaler = build_rnn_sequences(df, player, stat, threshold, features, window=5)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    nn = NeuralNetwork(X_train, X_test, y_train, y_test)
    nn.build_model(input_shape=(X.shape[1], X.shape[2]))
    nn.train()
    nn.evaluate()        


C:\Users\Brian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 5, 64)          │        18,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │           825 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,955 (124.82 KB)

 Trainable params: 31,955 (124.82 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 488ms/step - accuracy: 0.5441 - loss: 0.6947 - mean_absolute_error: 0.5007 - val_accuracy: 0.6429 - val_loss: 0.6865 - val_mean_absolute_error: 0.4965
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.6106 - loss: 0.6883 - mean_absolute_error: 0.4975 - val_accuracy: 0.7857 - val_loss: 0.6820 - val_mean_absolute_error: 0.4942
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.6779 - loss: 0.6821 - mean_absolute_error: 0.4942 - val_accuracy: 0.7857 - val_loss: 0.6774 - val_mean_absolute_error: 0.4917
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.6466 - loss: 0.6775 - mean_absolute_error: 0.4916 - val_accuracy: 0.7857 - val_loss: 0.6726 - val_mean_absolute_error: 0.4890
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.6779 - loss: 0.6722 - mean_absolute_error: 0.4885 - val_accuracy: 0.7857 - val_loss: 0.6668 - val_mean_absolute_error: 0.4858
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - acc

Model saved as 'rnn_player_model.h5'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.7143 - loss: 0.5509 - mean_absolute_error: 0.3651
Loss: 0.5509353280067444, Accuracy: 0.7142857313156128, MAE: 0.36507096886634827
